In [6]:
import torch
import torch.nn as nn
import math
# torch中变量封装函数Variable.
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F

In [4]:
emb = nn.Embedding(20, 5)
input = torch.LongTensor([[1,3,16,7], [10,9,5,2]])
emb(input)

tensor([[[ 0.9090, -1.7135,  0.9683,  0.7592,  1.4060],
         [-0.1570,  0.4284,  1.4959, -0.0457, -0.2729],
         [ 1.6651, -0.9901, -0.4175,  0.6405, -1.6994],
         [ 1.0410, -0.2590,  0.4833, -0.3428, -0.0436]],

        [[ 1.5341,  0.2422,  1.5804, -0.5379,  1.4425],
         [ 0.0678,  0.1272,  0.2976, -0.3707, -1.2650],
         [-1.0305, -1.4155,  1.1561, -0.0900,  1.3504],
         [-2.2858,  0.2465,  0.2565, -0.9511, -1.1107]]],
       grad_fn=<EmbeddingBackward0>)

In [6]:
emb = nn.Embedding(20, 3, padding_idx=0)
input = torch.LongTensor([[0,3,16,0], [10,0,5,2]])
emb(input)

tensor([[[ 0.0000,  0.0000,  0.0000],
         [-0.3544,  1.3096, -1.5186],
         [-0.0525,  0.5067, -1.0424],
         [ 0.0000,  0.0000,  0.0000]],

        [[-0.0766, -0.2262,  0.1055],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.7510,  0.4329, -0.4989],
         [-0.9452,  0.2293, -0.4351]]], grad_fn=<EmbeddingBackward0>)

In [12]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        """类的初始化函数，有两个參数，d_model：指词嵌入的维度，vocab：指词表的大小."""
        # 接着就是使用super的方式指明继承nn.Module的初始化函数，我们自己实现的所有层都会这样
        super(Embeddings, self).__init__()
        # 之后就是调用nn中的预定义层Embedding，获得一个词嵌入对象self.lut
        self.lut = nn.Embedding(vocab, d_model)
        # 最后就是将d_model传入类中
        self.d_model = d_model
        
    def forward(self, x):
        """可以将其理解为该层的前向传播逻辑，所有层中都会有此函数
        当传给该类的实例化对象參数时，自动调用该类函数
        参数x：因为Embedding层是首层，所以代表输入给模型的文本通过词汇映射后的张量"""
        # 将x传给self.lut井与根号下self.d_mode1相乘作为结果返回, 缩放
        return self.lut(x) * math.sqrt(self.d_model)

In [6]:
# 定义位置编码器类，我们同样把它看做一个层，因此会继承nn.Module 口
class PositionalEncoding(nn. Module):
    def __init__(self, d_model, dropout, max_len=5000):
        """位置编码器类的初始化函数，共有三个參数，分别是d_model:词嵌入维度，
        dropout:置0比率，max_len:每个句子的最大长度"""
        super(PositionalEncoding, self).__init__()
        # 实例化nn中预定义的Dropout层，并将dropout传入其中，获得对象self.dropout
        self.dropout = nn.Dropout(p=dropout)
        # 初始化一个位置编码矩阵，它是一个0阵，矩阵的大小是max_len x d_model.
        pe = torch.zeros(max_len, d_model)
        # 初始化一个绝对位置矩阵，在我们这里，词汇的绝对位置就是用它的索引去表示．
        # 所以我们首先使用arange方法获得一个连续自然数向量，然后再使用unsqueeze方法拓展向量维度
        # 又因为参数传的是1，代表矩阵拓展的位置，会使向量变成一个max-len x 1 的矩阵，
        position = torch.arange(0, max_len).unsqueeze(1)
        # 绝对位置矩阵初始化之后，接下来就是考虑如何将这些位置信息加入到位置编码矩阵中，
        # 最简单思路就是先将max_1en x 1的绝对位置矩阵，变换成max_len x d_model形状，然后覆盖原来的初始位置编码矩阵即可，
        # 要做这种矩阵变换，就需要一个1xd_model形状的变换矩阵div_term，我们对这个变换矩阵的要习除了形状外
        # 还希望它能够将自然数的绝对位置编码缩放成足够小的数字，有助于在之后的梯度下降过程中更快的收敛，这样我们就可以开始初始
        # 首先使用arange获得一个自然数矩阵，但是细心的同学们会发现，我们这里并没有按照预计的一样初始化一个1×d_model的矩阵
        # 而是有了一个跳跃，只初始化了一半即1xd_model/2 的矩阵。为什么是一半呢，其实这里并不是真正意义上的初始化了一半的矩阵
        # 我们可以把它看作是初始化了两次，而每次初始化的变换矩阵会做不同的处理，第一次初始化的变换矩阵分布在正玄波上，第二次
        # 并把这两个矩阵分别填充在位置编码矩阵的偶数和奇数位置上，组成最终的位置编码矩阵．
        div_term = torch.exp(torch.arange(0, d_model, 2) * 
                            -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        # 这样我们就得到了位置编码矩阵pe，pe现在还只是一个二维矩阵，要想和embedding的输出
        # 就必须拓展一个维度，所以这里使用unsqueeze拓展维度．
        pe = pe.unsqueeze(0)
        # 最后把pe位置编码矩阵注册成模型的buffer，什么是buffer呢，
        # 我们把它认为是对模型效果有帮助的，但是却不是模型结构中超參数或者參数，不需要随着优化步骤进行更新
        # 注册之后我们就可以在模型保存后重加载时和模型结构与参数一同被加载．
        self.register_buffer('pe', pe)

    def forward(self, x):
        """forward函数的参数是x，表示文本序列的词嵌入表示"""
        # 在相加之前我们对pe做一些适配工作， 将这个三维张量的第二维也就是句子最大长度的那一维将切
        # 因为我们默认max-1en为5000一般来讲实在太大了，很难有一条句子包含5800个词汇，所以要进行
        # 最后使用Variable进行封装，使其与x的样式相同，但是它是不需要进行梯度求解的，因此把requ
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        # 最后使用self.dropout对象进行"丢弃"操作，并返回结果.
        return self.dropout(x)

In [7]:
m = nn.Dropout(p=0.2)
input =  torch.randn(4, 5)
output = m(input)
output

tensor([[ 1.1562, -0.6876,  0.0424,  0.0000,  1.2023],
        [-3.2916, -1.2298,  0.3103, -0.0000, -2.2196],
        [ 2.3992, -3.1739,  1.1517,  0.4525, -1.0792],
        [ 0.6800, -2.0477,  0.4588,  0.7145,  0.0000]])

In [8]:
x = torch.tensor([1, 2, 3, 4])
torch.unsqueeze(x, 0)


tensor([[1, 2, 3, 4]])

In [9]:
torch.unsqueeze(x, 1)

tensor([[1],
        [2],
        [3],
        [4]])

In [13]:
# 词嵌入维度是512维度
d_model = 512

vocab = 1000

# 置0比率为0.1
dropout = 0.1

# 句子最大长度
max_len = 60

# 输入x是Embedding层的输出张量，形状是2×4×512
x = Variable(torch.LongTensor([[100, 2, 421, 500], [491, 999, 1, 220]]))
emb = Embeddings(d_model, vocab)
embr = emb(x)


In [14]:
embr.shape

torch.Size([2, 4, 512])

In [15]:
x = embr
pe = PositionalEncoding(d_model, dropout, max_len)
pe_result = pe(x)
print("pe_result:", pe_result)

pe_result: tensor([[[ 37.3150,  -4.8123,  -1.1419,  ...,   8.3940, -16.7525,  45.9313],
         [  0.0000,  11.2654,  27.7836,  ..., -17.2091, -28.4949,  -1.8657],
         [-51.7508, -18.5505,   3.6940,  ..., -11.0372, -21.4976,  10.9296],
         [ -6.9911,  14.9293, -36.5244,  ...,   0.0000,  -0.0000,  -6.6375]],

        [[-36.4800, -45.4464,  -8.2772,  ...,  26.8458,  18.3403, -20.4179],
         [-13.0741, -18.5222,  16.2227,  ...,  11.7240,   0.0000, -27.1928],
         [ -0.0000, -11.4216, -27.2937,  ...,  -9.3008,   0.0000, -25.8826],
         [  0.7426,  55.6398, -39.4706,  ..., -40.1964,  27.6500,  32.6713]]],
       grad_fn=<MulBackward0>)


In [16]:
pe_result.shape

torch.Size([2, 4, 512])

In [ ]:
# # 第一步设置一个画布
# plt.figure(figsize=(15, 5)）
# # 实例代PositionalEncoding类对象，词嵌入维度给20，置零比率设置为0
# pe = Positiona lEncoding(20, 0)
# # 向pe中传入一个全零初始化的x，相当于展示pe
# y = pe(Variable(torch.zeros(1, 100, 20))）
# plt.plot(np.arange(100), y[0, :])

In [2]:
def subsequent_mask(size):
    """生成向后道掩的掩码张量，參数size是掩码张量最后两个维度的大小，它的最后两维形成一个方阵"""
    # 在函数中，首先定义掩码张量的形状
    attn_shape = (1, size, size)
    # 然后使用np.ones方法向这个形状中添加1元素，形成上三角阵，最后为了节约空间，
    # 再使其中的数据类型变为无符号8位整形unit8
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    # 最后将numpy类型转化为torch中的tensor，内部做一个1 的操作，
    # 在这个其实是做了一个三角阵的反转，subsequent_mask中的每个元素都会被1减，
    # 如果是0，subsequent.mask中的该位置由0变成1
    # 如果是1，subsequent.mask中的该位置由1变成0
    return torch.from_numpy(1 - subsequent_mask)

In [3]:
np.triu([[1,2,3], [4,5,6],[7,8,9],[10,11,12]], k=-1)

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 0,  8,  9],
       [ 0,  0, 12]])

In [4]:
np.triu([[1,2,3], [4,5,6],[7,8,9],[10,11,12]], k=0)

array([[1, 2, 3],
       [0, 5, 6],
       [0, 0, 9],
       [0, 0, 0]])

In [5]:
np.triu([[1,2,3], [4,5,6],[7,8,9],[10,11,12]], k=1)

array([[0, 2, 3],
       [0, 0, 6],
       [0, 0, 0],
       [0, 0, 0]])

In [7]:
def attention(query, key, value, mask=None, dropout=None):
    """注意力机制的实现, 输入分别是query, key, value, mask: 掩码张量,
        dropout是nn.Dropout层的实例化对象, 默认为None"""
    # 在函数中，首先取query的最后一维的大小，一般情况下就等同于我们的词嵌入维度，命名为d_k
    d_k = query.size(-1)
    # 按照注意力公式，将query与key的转置相乘，这里面key是将最后两个维度进行转置，再除以缩放系数
    # 得到注意力得分张量scores
    scores = torch. matmul(query, key.transpose(-2,-1)) / math.sqrt (d_k)
    # 接着判断是否使用掩码张量
    if mask is not None:
    # 使用tensor的masked_fi11方法，将掩码张量和scores张量每个位置一一比较，如果掩码张量处理
    # 则对应的scores张量用-1e9这个值来替换，如下演示
        scores = scores.masked_fil1(mask == 0,-1e9)
    # 对scores的最后一维进行softmax操作，使用F.softmax方法，第一个参数是softmax对象，第二个
    # 这样获得最终的注意力张量
    p_attn = F.softmax(scores, dim =-1)
    #之后判断是否使用dropout进行隨机置e
    if dropout is not None:
    # 将p_attn传入dropout对象中进行'丢弃"处理
        p_attn = dropout(p_attn)
    # 最后，根据公式将p-attn与value张量相乘获得最终的query注意力表示，同时返回注意力张量
    return torch.matmul(p_attn, value), p_attn